##  estimate phenology from gimms ndvi

1. process raw ndvi data: choose site covered by plant, replace negative ndvi, savgol-filter

2. fit model to get daily ndvi sequence

3. get phenology using threshold or change point


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob
import dask

In [ ]:
import savitzky_golay
import numpy.polynomial.polynomial as poly
import scipy.interpolate as spi
from dask.diagnostics import ProgressBar

In [ ]:
def read_ndvi_data(paths):
    ndvi = []
    for file in paths:
        print(file)
        
        with xr.open_dataset(file,chunks=({'lat':1000,'lon':100})) as ndvi_i:
            ndvi.append(ndvi_i['ndvi'])
        
    ndvi_data = xr.concat(ndvi, dim = 'time')
    ndvi_data = ndvi_data.sortby('time')
        
    ndvi_data = ndvi_data.chunk({'lat':1000, 'lon':100, 'time':-1})
        
    return ndvi_data

In [ ]:
def dbl_logistic_model(p, doys):
    y = p[0] + p[1] * (1./(1+np.exp(p[2]*(doys-p[3]))) + 
                      1./(1 + np.exp(-p[4]*(doys - p[5]))))
    return y

In [ ]:
def mismatch_function(p, pheno_func, data, days):
    output = []
    
    fitness = lambda p, data_in, days: data_in - pheno_func(p, days)
    
    oot = fitness( p , data, days)
    [output.append(x) for x in oot]
    
    return np.array(output).squeeze()

In [ ]:
def fit_double_logistic(data, t_axis, xinit = None):
    
    from scipy.optimize import leastsq
    
    pheno_func = dbl_logistic_model
    
    n_params = 6
    if xinit is None:
        xinit = [.5,] * n_params
        xinit[0] = data.min()
        xinit[1] = data.max() - data.min()
        xinit[2] = 0.19
        xinit[3] = 120
        xinit[4] = 0.13
        xinit[5] = 260
        
    (xsol , msg) = leastsq(mismatch_function, xinit, maxfev=1000000,
                           args = (pheno_func, data ,t_axis))
    
    ax = pheno_func( xsol, np.arange(1,366))
    
    return (xsol, msg, ax)
        

In [ ]:
def piece_logistic(p,doys):
    y = p[0] + (p[1] - p[0])/(1+np.exp(p[2]*(p[3]-doys)))
    
    return y

In [ ]:
def fit_piece_logistic( data, t_axis, xinit = None, season = 'sos'):
    from scipy.optimize import leastsq
    
    pheno_func = piece_logistic
    n_params = 4
    
    if xinit is None:
        xinit = [.5] * n_params
        
        xinit[0] = data.min()
        xinit[1] = data.max()
        xinit[2] = 0.15
        
        if season == 'sos':
            xinit[3] = 120
        else:
            xinit[3] = 250
        
    (xsol, msg) = leastsq( mismatch_function,xinit, maxfev= 1000000,
                         args = (pheno_func, data, t_axis))
    
    if season == 'sos':
        ax = pheno_func(xsol, np.arange(1,t_axis.max()))
    else:
        ax = pheno_func(xsol, np.arange(t_axis.min(),t_axis.max()))
    return (xsol, msg, ax)

In [ ]:
def get_ndvi_phenology(ndvi_data, ndvi_mean):
    
    phenos = np.full(8, np.nan)
    
    ## check data of points
    if np.sum(np.isnan(ndvi_data)) < 12 and ndvi_mean > 0.1 :
        
        #doy = np.arange(8,365,15)  ## gimms ndvi
        doy = np.arange(8,365,16)  ## modis 16d ndvi
        ndvi_data = np.nan_to_num(ndvi_data, nan = ndvi_mean)
        growing_season = np.mean(ndvi_data[10:16])
        non_growing_1 = np.mean(ndvi_data[np.arange(6)])
        non_growing_2 = np.mean(ndvi_data[np.arange(20,len(ndvi_data))])
        non_growing_season = np.mean([non_growing_1,non_growing_2])
        
        doy_mid_raw = doy[ndvi_data.argmax()]
        is_mid = doy_mid_raw > 150 and doy_mid_raw < 270
        
        if growing_season > (1.2*non_growing_season)  and is_mid:
            
            ## SG-filter
            ndvi_data = np.asarray(ndvi_data[:len(doy)])
            ndvi_data_sg = savitzky_golay.savitzky_golay(ndvi_data,15,4)
            
            #plt.plot(ndvi_data_sg)
            ## divide into spr_data and aut_data
            index_mid = ndvi_data_sg.argmax()
            doy_mid = doy[index_mid]
            
            if doy_mid >160 and doy_mid < 220: 
                doy_spr = doy[:index_mid]
                doy_aut = doy[index_mid:]

                data_sg_spr = ndvi_data_sg[:index_mid]
                data_sg_aut = ndvi_data_sg[index_mid:]
            
            else:
                doy_spr = doy[doy <= 200]
                doy_aut = doy[doy > 170]
            
                data_sg_spr = ndvi_data_sg[doy <= 200]
                data_sg_aut = ndvi_data_sg[doy > 170]
            
            ## method 1: poly 50%
            poly_model = poly.polyfit(doy,ndvi_data_sg,6)
            poly_ndvi = poly.polyval(np.arange(1,366),poly_model)
            
            poly_ratio = (poly_ndvi- poly_ndvi.min()) / (poly_ndvi.max() - 
                                              poly_ndvi.min())
            
            if len(poly_ratio) > 0:
                phenos[0] = np.asarray(poly_ratio > 0.5).nonzero()[0][0]
                phenos[4] = np.asarray(poly_ratio > 0.5).nonzero()[0][-1]    
            
            ## method 2: spline 50%
            spi_spline = spi.splrep(doy,ndvi_data_sg,k=3)
            result_spline_3 = spi.splev(np.arange(1,366),spi_spline)

            ratio_spline = (result_spline_3 - result_spline_3.min()) / (
                result_spline_3.max() - result_spline_3.min())
            
            if len(np.asarray(ratio_spline > 0.5).nonzero()[0]) > 0:
                phenos[1] = np.asarray(ratio_spline > 0.5).nonzero()[0][0]
                phenos[5] = np.asarray(ratio_spline > 0.5).nonzero()[0][-1]
    
            ## method 3: dlog
            dlog_model = fit_double_logistic(ndvi_data_sg,doy)    
            
            dbl_fitted_ndvi = dlog_model[2]
            
            if len(dbl_fitted_ndvi) > 0:
                dbl_ndvi_1der = np.diff(dbl_fitted_ndvi)
            
                phenos[2] = np.arange(1,365)[dbl_ndvi_1der.argmax()]
                phenos[6] = np.arange(1,365)[dbl_ndvi_1der.argmin()]
    
            ## method 4: plog
            plog_spr = fit_piece_logistic(data_sg_spr, doy_spr)
            plog_aut = fit_piece_logistic(data_sg_aut, doy_aut, season = 'aut')
            
            if len(plog_spr[2]) > 0:
                plog_fitted_ndvi_spr = plog_spr[2]
                plog_spr_1der = np.diff(plog_fitted_ndvi_spr)
                
                phenos[3] = np.arange(1,doy_spr.max())[plog_spr_1der.argmax()]
                
            if len(plog_aut[2]) > 0:
                plog_fitted_ndvi_aut = plog_aut[2]
                plog_aut_1der = np.diff(plog_fitted_ndvi_aut)
            
                phenos[7] = np.arange(doy_aut.min(),366)[plog_aut_1der.argmin()]

    
    return phenos


In [ ]:
def ndvi_pheno(x,ndvi_mean):
    return xr.apply_ufunc(
        get_ndvi_phenology,
        x,
        ndvi_mean,
        input_core_dims = [['time'],[]],
        output_core_dims = [['pheno']],
        dask_gufunc_kwargs = {'output_sizes':{'pheno':8}},
        vectorize = True,
        dask = 'parallelized',
        output_dtypes= [float]
    )

In [ ]:
start_year = 2001
end_year = 2020
lat_lon = '-75-70'

In [ ]:
ndvi_path = glob.glob(r'D:/aithc/phd/data/modis_ndvi/modis_preprocessed/*'+lat_lon+'.nc')

In [ ]:
ndvi_data = read_ndvi_data(ndvi_path)

In [ ]:
ndvi_data

In [ ]:
ndvi_mean = ndvi_data.mean(dim = 'time', skipna =True)

# test functions

In [ ]:
ndvi_year = ndvi_data.groupby('time.year')[2001]

In [ ]:
point_data = ndvi_year[100,100,:].values
point_data

get_ndvi_phenology(point_data, 0.6)

# get phenology by year

In [ ]:
for year in range(start_year, end_year+1):
    
    print('now calculting....',year)
    
    ndvi_year = ndvi_data.groupby('time.year')[year]
    
    result = ndvi_pheno(ndvi_year, ndvi_mean)
    
    with ProgressBar():
        result_phenos = result.compute()
        
    pheno_list = year*100 + np.asarray([11,12,13,14,21,22,23,24])
    
    result_phenos['pheno'] = pheno_list
    result_phenos.name = 'phenology'
    
    result_phenos.to_netcdf(path = 'D:/aithc/phd/data/modis_ndvi/pheno/'+str(year)+lat_lon+'.nc',
                           encoding = {'phenology':{'dtype':'int16','_FillValue':-9999,
                                                   'zlib':True,'complevel':9}})